Разработана гибридная рекомендательная система на основе каскада:
1. сначала определим фильмы, которые наиболее близки к фильмам, которым пользователь поставил оценку 5. Для каждого фильма выделим по 10 близких фильмов 
2. далее для отобранных на 1 этапе применим KNNWithMeans для определения фильмов с наивысшей оценкой, которую мог бы поставить пользователь

In [1]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader

from scipy.spatial.distance import correlation

import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [3]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

Определим нужные функции

In [4]:
#Функция получает на вход userid и датасет с данными по оценкам фильмов
#Функция возвращает список фильмов, которым указанный пользователь поставил оценку 5
def ret_user_fav_films(userid, dataset):
    return dataset.loc[(dataset['userId'] == userid) & (dataset['rating'] == 5.0)].movieId.tolist()

#Функция получает на вход датасет с данными по оценкам фильмов
#Функция возвращает словарь векторов с оценками для каждого фильма
def movie_vectorizer(dataset):
    qty_users = dataset.userId.unique().shape[0]
    movie_vector = {}

    for movie, group in dataset.groupby('movieId'):
        movie_vector[movie] = np.zeros(qty_users)
    
        for i in range(len(group.userId.values)):
            u = group.userId.values[i]
            r = group.rating.values[i]
            movie_vector[movie][int(u - 1)] = r
     
    return movie_vector  

#Функция получает на вход список фильмов пользователя с оценками 5, словарь векторов фильмов, 
#а также количество подбираемых рекомендаций для каждого пользовательского фильма
#Функция возвращает фильмов, предварительно рекомендованных пользователю. 
#На основе этого списка будем делать дальнейшую рекомендацию
def movies_recommend(my_fav_films, vector, size=10):
    full_movies_list = []
    for my_fav_film in my_fav_films:
        titles = []
        distances = []
        for key in vector.keys():
            if key == my_fav_film:
                continue
                
            titles.append(key)
            distances.append(correlation(vector[my_fav_film], vector[key]))
        
        best_indexes = np.argsort(distances)[:size]
        best_movies = [(titles[i]) for i in best_indexes]
        full_movies_list.extend(best_movies)

    return list(set(full_movies_list) - set(my_fav_films))

#Функция получает на вход датасет, userid пользователя и количество фильмов для рекомендации
#Функция возвращает список рекомендованных пользователю фильмов

def make_rec(dataset, user_id, size=5):
    mov_id = []
    est = []
    for mov in dataset.iid.unique():
        mov_id.append(mov)
        est.append(algo.predict(uid=user_id, iid=mov).est)

        best_ind = np.argsort(est)[-size:]
        best_mov = [(mov_id[i]) for i in best_ind]
    return best_mov



In [5]:
#Получим список фильмов пользователя
user_films = ret_user_fav_films(3.0, movies_with_ratings)
#Построим словарь векторов фильмов
movie_vector = movie_vectorizer(movies_with_ratings)
#Получим список рекомендованных пользователю фильмов
rec_movies_list = movies_recommend(user_films, movie_vector, 10)
#выделим из всего датасета данные по рекомменлованным фильмам для дальнейшей обработки
user_rec_movies = movies_with_ratings.loc[movies_with_ratings.movieId.isin(rec_movies_list)]

In [6]:
#Подготовим данные для модели и обучим модель
dataset = pd.DataFrame({
    'uid': user_rec_movies.userId,
    'iid': user_rec_movies.movieId,
    'rating': user_rec_movies.rating
})

reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))
data = Dataset.load_from_df(dataset, reader)

trainset = data.build_full_trainset()

algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [7]:
#Предскажем рекоммендуемые фильмы для нужного пользователя
best_rec_mov = make_rec(dataset, 3)
best_rec_mov

[4180, 4813, 107771, 2529, 3740]

In [8]:
#выделим из общего датасета фильмов рекомендуемые пользователю фильмы
recommended_movies_for_user = movies.loc[movies.movieId.isin(best_rec_mov)]

In [9]:
recommended_movies_for_user

,movieId,title,genres
1905,2529,Planet of the Apes (1968),Action|Drama|Sci-Fi
2798,3740,Big Trouble in Little China (1986),Action|Adventure|Comedy|Fantasy
3110,4180,Reform School Girls (1986),Action|Drama
3522,4813,When Worlds Collide (1951),Sci-Fi
8334,107771,Only Lovers Left Alive (2013),Drama|Horror|Romance
